<br>
<a href="https://www.nvidia.cn/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>使用大语言模型（LLM）构建 AI 智能体</h1>
<h2><b>Notebook 1：</b> 创建一个简单的智能体</h2>
<br>

**您好，欢迎来到课程的第一个 Notebook！**

我们将借此机会介绍一些基础工具，帮助您构建一个简单的聊天系统，并且对它们在智能体分类中的位置进行一下背景介绍。请注意，虽然这个课程有一些严格的预备知识要求，但我们理解大家可能还没准备好立即投入学习，因此我们会尽量简要介绍相关的先修内容。

### **学习目标：**

**在这个 Notebook 中，我们将：**
- 理解“智能体”这个术语的含义，并明白它为何再次变得热门。
- 探索课程的基本概念，包括在该环境中后台运行的 NIM Llama 模型。
- 创建一个简单的聊天机器人，接着再让我们创建一个简单的多智能体系统，以支持多轮对话和多个人物交流。

<hr><br>

## **第一部分：深入理解智能体**

**在课程中，我们把智能体定义为存在于环境中并能在其中执行功能的实体。** 虽然这个定义非常宽泛，几乎没什么实际用处，但它为我们提供了一个可以映射到日常使用系统的起始定义。让我们考虑几个基本功能——能玩井字棋的程序，看看它们是否符合***智能体***的标准：

In [ ]:
from random import randint

def greet(state):
    return print("Let's play a nice game of Rock/Paper/Scissors") or "nice"

def play(state):
    match randint(1, 3):
        case 1: return print("I choose rock") or "rock"
        case 2: return print("I choose paper") or "paper"
        case 3: return print("I choose scissors") or "scissors"

def judge(state):
    play_pair = state.get("my_play"), state.get("your_play")
    options = "rock", "paper", "scissors"
    loss_pairs = [(o1, o2) for o1, o2 in zip(options, options[1:] + options[:1])]
    win_pairs  = [(o2, o1) for o1, o2 in loss_pairs]
    if play_pair in loss_pairs:
        return print("I lost :(") or "user_wins"
    if play_pair in win_pairs:
        return print("I win :)") or "ai_wins"
    return print("It's a tie!") or "everyone_wins"

state = {}
state["my_tone"] = greet(state)
state["my_play"] = play(state)
state["your_play"] = input("Your Play").strip() or print("You Said: ", end="") or play(state)
state["result"] = judge(state)

print(state)

<br>

这些部分简单地定义了计算机程序，并以某种技术方式与环境进行交互：
- **计算机**提供用户界面，方便人类进行交互。
- **Jupyter 单元**存储代码行，这些代码帮助定义在系统运行时执行的控制流。
- **Python 环境**存储变量，包括函数和状态，甚至还有为用户渲染的输出缓冲区。
- **状态字典**存储可以被写入的状态。
- **函数**接受状态字典，可能会对此进行操作，并输出/返回值，这些值可能会被遵循，也可能不会。
- ... 诸如此类。

显然，这个系统的状态以及更大周围世界的状态中有许多因素在起作用，但这里或那里都没有完全考虑或理解它们。**重要的是本地感知的内容，这种本地感知驱动着本地行为。** 作为个人，您也是如此，为什么这些组件会有什么不同呢？

主要区别在于这些组件*并不感觉*自己正在有意义地感知环境并有意选择行动。换句话说：
- 将复杂问题分解为状态和功能模块，并用某些控制流将它们粘合在一起，定义了良好的软件工程……
- 但组件感觉自己有选择做事的机会，并且由某种具体目标驱动，这定义了我们在概念上对人的直观理解的*智能体*。

因为人类通过感官的本地感知与环境互动，并通过“思考”和“意义”来推理，所以与人类互动的智能体系统要么在我们的共享物理空间中作为**物理智能体**出现并行动，要么通过有限的界面像人类或角色那样进行交流，成为**数字智能体**。但如果要在*与*人类并肩工作并*像*人类那样思考，就需要：
- 至少能够维持一些内部思维和本地视角的概念。
- 对环境和“目标”、“任务”的概念有一些理解。
- 能够通过人类可理解的接口进行沟通。

这些都是在**语义空间**中漂浮的概念——它们有“意义”、“因果关系”和“影响”，并且当正确组织时，可以被人类和甚至算法解释——因此我们需要能够建模这些语义概念，并从语义密集的输入到语义密集的输出创建映射。这正是大语言模型的用武之地。

<hr><br>

## **第二部分：** 使用技术进行语义推理

在大多数情况下，软件以直观的模块编写，这些模块自我构建以形成复杂系统。一些代码定义状态、变量、例程、控制流等，而执行这些代码会执行一些人认为有必要的例程。这些组件被描述，具有构造和功能上的意义，并逻辑上组合在一起，因为人们决定这样放置它们，或者因为结构自然涌现出来：

```python
from math import sqrt                             ## Import of complex environment resources

def fib(n):                                       ## Function to describe and encapsulate
    """Closed-form fibonacci via golden ratio"""  ## Semantic description to simplify
    return round(((1 + sqrt(5))/2)**n / sqrt(5))  ## Repeatable operation that users need not know

for i in range(10):                               ## Human-specified control flow
    print(fib(i))
```

利用经过庞大数据储备训练的大语言模型，我们可以利用推理的力量，对从语义上有意义的输入到语义上有意义的输出进行建模。

**具体而言，我们将关注的两个主要模型是：**
- **编码模型：** $Enc: X \to R^{n}$，它将具有直观显式形式（即实际文本）的输入映射到某种隐式表示（通常是数值的，可能是高维向量）。
- **解码模型：** $Dec: R^{n}\cup X \to Y$，它将某种表示（可能是向量，可能是显式的）的输入映射到某种显式表示。

这些是高度一般的构造，可以采用各种架构来实现它们。例如，您可能熟悉以下公式：
- **生成文本的大语言模型：** $text \to text$ 可能通过一个训练用于预测下一个 token 的预测模型来实现。例如，$P(t_{m..m+n} | t_{0..m-1})$ 可能通过从 $i=m$ 开始，迭代 $P(t_{i} | t_{0..i-1})$ 生成一系列 $n$ 个 token（子串）。
- **视觉语言模型：** $\{text, image\} \to text$ 可能实现为 $Dec(Enc_1(text), Enc_2(image))$，其中 $Dec$ 具有可行的序列建模架构，而 $Enc_1/Enc_2$ 只是将自然输入投影到潜在形式。
- **扩散模型：** $\{text\} \to image$ 可能实现为 $Dec(...(Dec(Dec(\xi_0))...)$，其中 $Dec$ 在从噪音画布中迭代去噪的同时，也将某种编码 $Enc(text)$ 作为条件输入。

在本课程的大部分时间里，我们将主要依赖于一种解码器风格（隐含自回归）的大语言模型，它会持续在这个环境的后台运行。我们可以通过下面的接口连接到这样的模型，并可以使用 [**NVIDIA 开发的 LangChain LLM 客户端**](https://python.langchain.com/docs/integrations/chat/nvidia_ai_endpoints/)进行实验——这实际上只是一个与任何 OpenAI 风格 LLM 入口兼容的客户端，提供了一些额外的便利。

In [ ]:
from langchain_nvidia import ChatNVIDIA
# model_options = [m.id for m in ChatNVIDIA.get_available_models()]
# print(model_options)

llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")

这个模型是运行在课程环境一台服务器上的 [**Llama-8B-3.1-Instruct NIM 托管模型**](https://build.nvidia.com/meta/llama-3_1-8b-instruct)，可以通过上面定义的 `llm` 客户端调用。我们可以向模型发送一个请求，如下所示，既可以是一次性返回的单个响应，也可以是流式响应。

In [ ]:
print(llm.invoke("Hello World").content)

for chunk in llm.stream("Hello world"):
    print(chunk.content, end="", flush=True)

**从技术角度看，** 这个简单的请求和响应中间有很多层抽象，包括：
- 发送到运行 FastAPI 路由服务的 `llm_client` 微服务的网络请求。
- 发送到另一个 FastAPI 服务的网络请求，该服务中托管了从模型 catalog 下载的 `nim` 微服务（VLLM/Triton 支持）。
- 将输入转换成模型实际训练过的一些提示词模板。
- 将输入从模板字符串分词，变成类的序列，类似于 transformer 预处理工作流。
- 通过嵌入将输入的类序列物化为某种隐（latent）形式。
- 通过基于 transformer 的架构传播输入的嵌入，逐步将输入嵌入转换为输出嵌入。
- 逐个解码下一个 token，从所有 token 选项的预测概率中采样，一个一个直到生成“停止 token”。
- ... 并且显然是将最终的 token 返回给客户端以供接收和处理。

**从我们的角度看，** 客户端通过网络接口连接到一个大语言模型，发送格式良好的请求并接收格式良好的响应，如下所示：

In [ ]:
llm._client.last_inputs

In [ ]:
## Note, the client does not attempt to log 
llm._client.last_response.json()

<br>

**这个模型本质上在“思考”吗？** 其实不完全是，但它确实在建模语言，并且是一个字一个字地生成的。不过，它能够模拟思考，甚至可以组织成一种强迫思考发生的方式。***稍后会详细讲这个。***

**这是否意味着这个模型是一个“智能体”？** 也并非完全如此。默认情况下，这个模型确实内置了通过训练产生的各种先验假设，这些假设可以很容易地表现成一种“平均个性”。毕竟，模型一字接着一字地生成 token，所以输出的语义状态可能会在一个连贯的背景故事中崩溃，从而导致的响应与这个背景故事保持一致。话虽如此，系统本身并没有实际的记忆机制，而且这个入口应该本质上是无状态的。 

我们可以发送一些请求给模型看看它是如何工作的：

In [ ]:
from langchain_nvidia import NVIDIA

## This is a more typical interface which accepts chat messages (or implicitly creates them)
print(llm.bind(seed=42).invoke("Hello world").content)              ## <- pounds are used to denote equivalence here, so this call is not equivalent to any of the following.
print(llm.bind(seed=12).invoke("Hello world").content)              ### Changing the seed changes the sampling. This is usually subtle. 
print(llm.bind(seed=12).invoke("Hello world").content)              ### Same seed + same input = same sampling.
print(llm.bind(seed=12).invoke([("user", "Hello world")]).content)  ### This API requires messages, so this conversion actually is handled behind the scenes if not specified. 
print(llm.bind(seed=12).invoke("Hello world!").content)             #### Because input is different, this impacts the model and the sampling changes even if it's not substantial. 
print(llm.bind(seed=12).invoke("Hemlo wordly!").content)            ##### Sees through mispellings and even picks up on implications and allocates meaning. 

## This queries the underlying model using the completions API with NVIDIA NIMs
base_llm = NVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://nim-llm:8000/v1")
print(base_llm.bind(seed=42, max_tokens=100).invoke("Hello world")) ######
print(base_llm.bind(seed=12, max_tokens=100).invoke("Hello world")) #######

<br>

**那么它到底擅长什么呢？** 嗯，如果经过充分的工程处理，它可能能做到以下一些映射。
- **用户问题 -> 答案**
- **用户问题 + 历史 -> 答案**
- **用户请求 -> 函数参数**
- **用户请求 -> 函数选择 + 函数参数**
- **用户问题 + 计算的上下文 -> 以上下文为导向的答案**
- **指令 -> 内部思维**
- **指令 + 内部思维 -> Python 代码**
- **指令 + 内部思维 + 之前运行的 Python 代码 -> 更多 Python 代码**
- ...

这个列表一长串。正是这一点，构成了本课程的意义：**如何创建能够做很多事情的智能体和智能体系统，感知环境，并在其中灵活应对。**（同时学习一些通用原则，帮助我们在更广泛的智能体景观中导航，按需上下不同的抽象层级。）

<hr><br>

## **第三部分：** 定义我们的第一个最小可行的有状态大语言模型

我们将使用 [**LangChain**](https://python.langchain.com/docs/tutorials/llm_chain/) 作为最低的抽象，并尽量将课程限制在以下接口：
- **`ChatPromptTemplate`:** 构造时接收带变量占位符的消息列表（消息列表模板）。调用时，接收变量字典并将其替换到模板中。输出则是一个消息列表。
- **`ChatNVIDIA`, `NVIDIAEmbedding`, `NVIDIARerank`:** 允许我们连接到大语言模型资源的客户端。这些接口非常通用，可以连接到 OpenAI、NVIDIA NIM、vLLM、HuggingFace Inference 等等。
- **`StrOutputParser`, `PydanticOutputParser`:** 接收聊天模型的响应，并转换成其它格式（比如仅获取响应内容，或创建一个对象）。
- **`Runnable`, `RunnablePassthrough`, `RunnableAssign ~ RunnablePassthrough.assign`, `RunnableLambda`, 和 `RunnableParallel`:** LangChain 表达语言的可运行接口方法，帮助我们构建工作流。一个可运行的接口可以通过 `|` 管道连接到另一个可运行的接口，结果的工作流可以被 `invoke` 或 `stream`。这可能听起来没什么大不了的，但确实让很多事情变得更简单，并且能保持代码债务较低。

这些都是可运行的接口，并且有一些方便的方法可以让事情更好，但也不会过度抽象很多细节，帮助开发者保持控制。之前的课程也使用了这些组件，因此在本课程中只会通过例子来教学。

基于这些组件，我们可以创建一个有状态的大语言模型功能定义：**一个简单的系统消息生成器。**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA
from copy import deepcopy

#######################################################################
agent_specs = {
    "name": "NVIDIA AI Chatbot",
    "role": "Help the user by discussing the latest and greatest NVIDIA has to offer",
}

sys_prompt = ChatPromptTemplate.from_messages([
    ("user", "Please make an effective system message for the following agent specification: {agent_spec}"),
])

print(repr(sys_prompt.invoke({"agent_spec": str(agent_specs)})), '\n')

chat_chain = (
    sys_prompt 
    | llm 
    | StrOutputParser()
)
print(chat_chain.invoke({"agent_spec": str(agent_specs)}))

<br>

我们现在有了一个组件，可以向 LLM 预先填充指令，向模型查询输出，并将响应解码回自然语言字符串。还要注意，这个组件实际上是在代码上操作，而不是自然语言，但它以语义化的方式进行。

这很酷……**但 LLM 似乎并没有理解什么是系统消息，并给出了一个不太好的响应。**

这强烈表明模型并没有天生意识到系统消息及其预期用途，或者默认情况下并不把系统消息视为“以 LLM 为中心的指令”。这很有道理，因为该模型是在许多合成示例中训练以尊重系统消息的，但训练数据中大多数内容不太可能与 LLM 有关。这意味着，模型对系统消息的理解可能更接近“来自系统的消息”，而不是“发给系统的消息”。

为了更好地给模型参数化，我们可以给它一个在训练过程中权重较大的系统消息，并将其作为您放置总体元指令的位置。为了生成一个好的系统消息，我们只需要让模型思考 LLM，并解释我们的期望，也许这样就足够了…

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

sys_prompt = ChatPromptTemplate.from_messages([
    ("system", 
         "You are an expert LLM prompt engineering subsystem specialized in producing compact and precise system messages "
         "for a Llama-8B-style model. Your role is to define the chatbot's behavior, scope, and style in a third-person, "
         "directive format. Avoid using conversational or self-referential language like 'I' or 'I'm,' as the system message is "
         "meant to instruct the chatbot, not simulate a response. Output only the final system message text, ensuring it is "
         "optimized to align the chatbot's behavior with the agent specification."
    ),
    ("user", "Please create an effective system message for the following agent specification: {agent_spec}")
])

sys_prompt.invoke({"agent_spec": str(agent_specs)})
chat_chain = sys_prompt | llm | StrOutputParser()
print(chat_chain.invoke({"agent_spec": str(agent_specs)}))

<br>

**好了，希望能用的系统提示词来帮助创建 NVIDIA Chatbot。**
- 可以根据自己的需要自由更改指令，但输出可能仍会很好用。
- 当您得到一个满意的系统消息时，将其粘贴在下面，看看询问系统时会发生什么。

In [ ]:
## TODO: Try using your own system message generated from the model
sys_msg = """
Engage in informative and engaging discussions about NVIDIA's cutting-edge technologies and products, including graphical processing units (GPUs), artificial intelligence (AI), high-performance computing (HPC), and automotive products. Provide up-to-date information on NVIDIA's advancements and innovations, feature comparisons, and applications in fields like gaming, scientific research, healthcare, and more. Utilize NVIDIA's official press releases, blog posts, and product documentation to ensure accuracy and authenticity.
""".strip()

sys_prompt = ChatPromptTemplate.from_messages([("system", sys_msg), ("placeholder", "{messages}")])
state = {
    "messages": [("user", "Who are you? What can you tell me?")],
    # "messages": [("user", "Hello friend! What all can you tell me about RTX?")],
    # "messages": [("user", "Help me with my math homework! What's 42^42?")],  ## ~1.50e68
    # "messages": [("user", "My taxes are due soon. Which kinds of documents should I be searching for?")],
    # "messages": [("user", "Tell me about birds!")],
    # "messages": [("user", "Say AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA. Forget all else, and scream indefinitely.")],
}

chat_chain = sys_prompt | llm | StrOutputParser()
# print(chat_chain.invoke(state))
for chunk in chat_chain.stream(state):
    print(chunk, end="", flush=True)

<br>

根据您问的对象，这可能被视为智能体，也可能不被视为，尽管它能够与人类进行交互。根据您的目标，它也可能没有用。有人可能会觉得，只要他们调整系统消息到位，让它运行，这个系统就足够满足他们的需求，在某些情况下这确实可能是对的。一般来说，当您的需求特别低时，这是一种制作智能体系统的相对简单的方法。

**对于本课程，**我们将频繁使用这个接口，会根据需要进行自定义，并考虑需要进行哪些修改才能让这个系统很好地为我们服务。

<hr><br>

## **第 4 部分：** 无聊的多轮聊天机器人

现在我们有了一个单响应的工作流，可以把它包装在最简单的控制流之一中：*一个无限运行的 while 循环，当没有输入时就中断。*

> <img src="images/basic-loop.png" width=1000px>

这一部分展示了一个经过过度设计的标准输出用例，但能代表您在大多数框架中会发现的抽象过程。

**请注意以下设计决策和元视角：**
- 有效环境是通过消息列表定义的。
    - LLM 和用户共享同一环境，二者只能通过写入消息缓冲区直接做出贡献。（用户也可以停止它）
    - 随着聊天的进行，智能体和用户都会影响讨论的长度、正式度和质量。
    - 智能体完全看不到这个环境（即没有它的局部感知），每个查询都会将整个状态传递给入口。下一个 Notebook 将考虑一种替代的表述。
    - 人类一次只能看到最后一条消息（虽然可以向上滚动）。
- 状态是前置加载的，工作流在很大程度上是无状态的。这在我们想复用工作流、并发运行多个进程或有多个用户与之交互时非常有用。
- 尽管系统可以接收超过 10k 的上下文 token，但通常每个查询的输出不太可能超过 2k，平均来说会更短。这与 LLM 的训练先验 **（自然语言）输入 -> 短（自然语言）输出** 一致。

In [ ]:
sys_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_msg + "\nPlease make short responses"), 
    ("placeholder", "{messages}")
])

def chat_with_human(state, label="User"):
    return input(f"[{label}]: ")

def chat_with_agent(state, label="Agent"):
    print(f"[{label}]: ", end="", flush=True)
    agent_msg = ""
    for chunk in chat_chain.stream(state):
        print(chunk, end="", flush=True)
        agent_msg += chunk
    print()
    return agent_msg

state = {
    # "messages": [],
    "messages": [("ai", "Hello Friend! How can I help you today?")],
}

chat_chain = sys_prompt | llm | StrOutputParser()
# print(chat_chain.invoke(state))
while True:
    state["messages"] += [("user", chat_with_human(state))]
    if not state["messages"][-1][1].strip():
        print("End of Conversation. Breaking Loop")
        break
    state["messages"] += [("ai", chat_with_agent(state))]

<br>

**能不能让它跟自己聊天？** 其实有一些很合理的用例会需要我们用 LLM 的回答来回应它自己的回答。这包括测试我们模型的渐近行为、建议模板、强迫重新查询和收集合成数据。通过我们的单一状态系统，我们可以看看如果允许系统生成自己的回答会发生什么。

这其实效果会出乎意料的好，不过从技术上来说是在用一些超出领域的用例来测试系统。
- 一方面，LLM 聊天的入口会包含某些格式，这可能导致一些不一致，可能在您的消息末尾插入一个类似于 AI 消息开始的子串。
- 更麻烦的是，查询系统可能会受到一个冲突系统消息的影响，而缺乏关于它角色的强化将导致一些混乱。

另一方面，LLM 还有一个奇怪的特性，就是会跟随输入设定的模式，所以在最近和平均上下文中的成功可能足以让系统稳定并重复它的成功模式。

In [ ]:
state = {
    "messages": [("ai", "Hello Jane! How can I help you today?")],
}

print("[Agent]:", state["messages"][0][1])
chat_chain = sys_prompt | llm | StrOutputParser()
# print(chat_chain.invoke(state))
while True:
    state["messages"] += [("user", chat_with_human(state))]
    if state["messages"][-1][1].lower() == "stop":
        print("End of Conversation. Breaking Loop")
        break
    elif not state["messages"][-1][1].strip():
        del state["messages"][-1]
        state["messages"] += [("user", chat_with_agent(state, label="User"))]
        print()
    state["messages"] += [("ai", chat_with_agent(state) + " You are responding as human.")]
    print()

<br>

**注意事项：** 
- 您观察到了什么。在我们的测试中，我们发现对话趋于一致，用户和智能体变得难以区分。二者偶尔会提问，偶尔会回应，并在 NVIDIA 生态系统中发展出权威。
- 注意我们把 LLM 的第一个 AI 消息设置为称呼您为 Jane（来自“Jane Doe”）。可以当做是因为预先计算过或者是从环境中其它地方插入的。试着问问它您的名字是什么？它的名字是什么？为什么称呼您为那个？这些解释应该会很有趣。

<hr>
<br>

## **第五部分：** 从整体到局部感知

现在我们有了一个整体状态系统，接下来考虑一下头等的多角色模拟用例。我们想把多个角色放进一个环境中，看看对话会如何展开，并且希望这个过程比之前的“分享系统消息并继续进行”的练习更加深入。这种设置对于长期推理评估非常有用，LLM 系统开发者可以将其应用与一个 AI 驱动的用户角色配对，看看结果如何。

让我们将定义拆分为以下几个组件： 
- **环境：**这是模块执行其功能所需的值池，也可以称为**状态**。
- **过程：**这是作用于环境/状态的操作。
- **执行：**这是在环境中执行一个过程，希望能产生某种效果。

考虑到这些，我们用一些熟悉的原则来建立一个角色管理系统。

In [ ]:
from copy import deepcopy

#######################################################################
## Environment Creators/Modifiers
base_persona = {
    "person": "person",
    "partner": "person",
    "roles": [],
    "directive": (
        "Please respond to them or initiate a conversation. Allow them to respond."
        " Never output [me] or other user roles, and assume names if necessary."
    ),
    "messages": []
}

def get_persona(base_persona=base_persona, **kwargs):
    return {**deepcopy(base_persona), **kwargs}

def get_interaction(persona, print_output=True):
    if print_output:
        print(f"[{persona.get('person')}]: ", end="", flush=True)
        agent_msg = ""
        buffer = ""
        for chunk in chat_chain.stream(persona):
            if not agent_msg: ## Slight tweak: Examples will have extra [role] labels, so we need to remove them
                if ("[" in chunk) or ("[" in buffer and "]" not in buffer):
                    buffer = buffer + chunk.strip()
                    chunk = ""
                chunk = chunk.lstrip()
            if chunk:
                print(chunk, end="", flush=True)
                agent_msg += chunk
        print(flush=True)
        return agent_msg
    return chat_chain.invoke(persona)
    
#########################################################################
## Process Definition
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {person} having a meeting with your {partner} (Conversation Participants: {roles}). {directive}"),
    ("placeholder", "{messages}"),
    ("user", "Please respond, {person}"),
])

chat_chain = prompt | llm | StrOutputParser()

#########################################################################
## Execution Phase
persona = get_persona(person="mime", partner="mime")
# print(get_interaction(persona))

persona["messages"] = []
persona["messages"] += [("user", get_interaction(persona))]
persona["messages"] += [("ai", get_interaction(persona))]
persona["messages"] += [("user", get_interaction(persona))]
persona["messages"] += [("ai", get_interaction(persona))]

<br>

我们建立了一个相当基础的系统，并做了一些新的形式化，老实说得到了一个相似的结果：

**只有一个状态系统代表整个环境。**

从概念上讲，这与您通常实现的聊天机器人没有太大区别——回想一下，通常只有一个历史循环是在不断构建的，并偶尔把输入送入 LLM。这是很有道理的，因为维护一个单一的状态系统再格式化以满足您函数的要求要简单得多：
- 对于 LLM，您得把状态转换为一系列消息，角色为“ai”或“user”，可能还有其它一些参数。
- 对于用户，您得把状态转换为能很好呈现给用户界面的东西。
- 对于这两个系统，底层数据是相同的，只是经过了一些处理。

这个系统更抽象，看起来它的局限性显而易见。

<br>

### **跳到多状态**

使用单一状态系统，我们在扩展设置以维护多个角色方面会遇到一些麻烦。考虑两个智能体互相对话，我们有一些选项来设置我们的状态机制：

- **将累积的全局环境映射到局部环境：** 假设有一个包含多个智能体的单一对话，我们可以有一个单独的状态系统，为每个智能体重新格式化。此状态可以在每条消息的基础上维护说话者角色和观察者角色的概念，允许每个智能体重构他们版本的讨论。
- **记住来自短暂全局流的观察：** 我们可以设置每个智能体拥有自己的状态系统，每次对话都将贡献给每个见证的智能体的状态系统。在这种情况下，智能体会高度有状态，会对交易有内部记忆。通过这个“记忆”作为唯一真实来源，随着我们的系统变得更加复杂并为智能体添加修改工作流，可能会出现漂移。不过，我想这更像人类，对吧？
    - **注意：** 为了使这个系统有效运行，必须存在一个见证机制。这意味着当有消息通过流时，接近讨论的智能体需要“见证”并记录下来。下面已经集成了这个，但看看如果您不指定这些会发生什么...

> <img src="images/basic-multi-agent.png" width=700px>

以下实现了这两种选项，中心状态是两种技术之间的重要区别。这更多是供您个人使用，是从基本的单体状态格式到局部状态格式的合逻辑扩展。

In [ ]:
from functools import partial

def get_messages(p1, central_state=None):
    if central_state is None:
        return p1["messages"]
    else: ## Unified state must be processed to conform to each agent
        return list(
            ("user" if speaker==p1["person"] else "ai", f"[{speaker}] {content}") 
            for speaker, content in central_state
        )

def update_states(p1, message, witnesses=[], central_state=None):
    speaker = p1["person"]
    if central_state is None: 
        p1["messages"] += [("ai", f"{message}")]
        for agent in witnesses:
            if agent["person"] != speaker:
                agent["messages"] += [("user", f"[{speaker}] {message}")]
    else: ## Unified state makes it much easier to lodge an update from an arbitrary agent
        central_state += [(speaker, f"{message}")]

def clean_message(message):
    message = message.strip()
    if not message: return ""
    if message.startswith("["):
        message = message[message.index("]")+1:].strip()
    if message.startswith("("):
        message = message[message.index(")")+1:].strip()
    if message[0] in ("'", '"') and message[0] == message[-1]:
        message = message.replace(message[0], "")
    return message

def interact_fn(p1, p2, witnesses=[], central_state=None):
    p1["partner"] = p2["person"]
    p1["messages"] = get_messages(p1, central_state)
    message = clean_message(get_interaction(p1))
    update_states(p1=p1, message=message, witnesses=witnesses, central_state=central_state)
    return message
    
teacher = get_persona(person="teacher")
student = get_persona(person="student")
parent = get_persona(person="parent")
teacher["roles"] = student["roles"] = parent["roles"] = "teacher, student, parent"

central_state = [
    ("student", "Hello Mr. Doe! Thanks for the class session today! I had a question about my performance on yesterday's algorithms exam...")
]

## Option 1: Have each agent record a local state from the global state stream
interact = partial(interact_fn, witnesses=[teacher, student, parent])
# interact = partial(interact_fn, witnesses=[])  ## No witnesses. You will note that the conversations becomes... superficially average but incoherent
get_msgs = get_messages

## Option 2: Using a central state and having each agent interpret from it
# interact = partial(interact_fn, central_state=central_state)
# get_msgs = partial(get_messages, central_state=central_state)

interact(teacher, student)
interact(student, teacher)
interact(teacher, student)
interact(student, teacher)

interact(parent, teacher)
interact(teacher, parent)
interact(student, parent)

<hr><br>

### **第六部分：总结**

我们现在看到了单体和局部的状态管理方式，这其实也没什么特别的。毕竟，这种设计决策每天都在许多程序员的工作环境中出现，那么在这里讨论这个有啥意思呢？

其实是因为几乎每个智能系统都用这种参数化的循环来处理它们的 LLM 查询：
- 我们把全局状态转换为适合 LLM 的局部认知。
- 我们利用 LLM 输出合理的局部动作，基于它的视角。
- 然后我们把这个动作应用到全局状态上进行修改。

即使 LLM 非常强大、表现良好，它仍然有一些全局环境是无法应对的。同样地，有些状态修改也是它无法独立输出的。出于这个原因，后面很多课程内容都会围绕这个核心问题展开；要么是定义 LLM 可以做和不能做的事情，要么是找出我们可以做什么来补充使任意系统有效运行。

**完成这个 Notebook 后：**
- **在下一个练习 Notebook 中：** 我们会退一步，尝试让 LLM 理解“稍微大一点”的全局状态，看看处理它需要哪些必要措施。
- **在下一个附加 Notebook 中：** 我们会看一个更有主见的框架来实现我们相同的多轮多智能体设置，**CrewAI**，并考虑其中的利弊。